In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import json
import folium

This part load the dataframe with the % of yes and clean the dataframe so that we have only the date in one column and another to have only the subject of the votation.

In [ ]:
#The dataframe with the votation data
df = pd.read_pickle("data/data.pkl")
df['Date'] = df['Votation'].map(lambda x : x.split(' ')[0])
#df['Votation'] = df['Votation'].map(lambda x : ' '.join(x.split(' ')[1:]))
df.head()

In [ ]:
#download all the page 
xls = pd.ExcelFile('data/Recommandations des partis.xls')
#the dataframe to fill with the info of the recommandation
recommend = pd.DataFrame()

#for every page with information from 2017 to 1981  :
for i in range ((2017 - 1981 + 1)) :
    #get the page i
    x = xls.parse(i)
    #change the name of the columns so that they will be easier to use 
    x.columns = range(len(x.columns))
    
    #We will need the parties and the numero of the votation, for that we use the fact that the line that have the date 
    #always begin by 'Parti 1)' and that the no of votation is 2 lines later
    base_nb = x[x.iloc[:, 0] == 'Parti 1)'].index[0]
    
    #We drop all the columns with only Nan
    x = x.dropna(axis=1, how='all')

    #concat the date, the No of votation and the conseil of vote using the fact that they are the only lines with Nan
    recommend_inter = pd.concat([x.iloc[base_nb:base_nb + 1], x.iloc[base_nb + 2:base_nb + 3], x.dropna()]).transpose()
    
    #this part change the name of the party so that even if there is little change in the name  (like had a 3) to the name)
    #we still have consistent name
    recommend_inter.iloc[0, 2:] = recommend_inter.iloc[0, 2:].map(lambda x : x.split(' ')[0])    
    
    #change the name of the columns so that they will be easier to use
    recommend_inter.columns = range(len(recommend_inter.columns))
    
    recommend_inter.iloc[:, 0] = recommend_inter.iloc[:, 0] + str(2017 - i)
    # make sure that 'no ###' are  made into 'No ###'
    recommend_inter.iloc[:, 1] = recommend_inter.iloc[:, 1].map(lambda x : x[0].upper() + x[1:], na_action='ignore')
    
    #get the lines that give the name of a votation knowing its No
    propositions = x[x.iloc[:, 0].str.contains('No ').fillna(False)]
    #change the name of the columns so that they will be easier to use
    propositions.columns = range(len(propositions.columns))
    
    #The next part is transforming the No of votation into name of votation
    
    #in some sheet this information is on only 1 cell so we split the information in 2 clls like the rest of the sheets 
    if (propositions.iloc[:, 1].isnull()).all():
        if (2017 - i == 1997):
            propositions.iloc[:, 1] = propositions[0].map(lambda x : x.split(' ')[1])
            propositions.iloc[:, 0] = propositions[0].map(lambda x : x.split(' ')[0].rstrip())
        else:
            propositions.iloc[:, 1] = propositions[0].map(lambda x : x.split(':')[1][1:])
            propositions.iloc[:, 0] = propositions[0].map(lambda x : x.split(':')[0].rstrip())
    
    
    dico_no_propos = propositions.dropna(1).set_index(0).to_dict()
    
    #some dico are inside a dictionarry {1:{true_dictionarry}}
    if (dico_no_propos.get(1) == None) :
        recommend_inter.iloc[:, 1] = recommend_inter.iloc[:, 1].map(dico_no_propos)
    else :
        recommend_inter.iloc[:, 1] = recommend_inter.iloc[:, 1].map(dico_no_propos[1])
        
    #rename column meaningfully
    recommend_columns = ['Date', 'Votation'] + list(recommend_inter.iloc[0][2:])
    recommend_inter.columns = recommend_columns
    
    #fill the date for the line that does not have it
    recommend_inter['Date'] = recommend_inter['Date'].fillna(method='ffill')
    #transpose so that we can join
    recommend_inter = recommend_inter.transpose()
    
    #delete duplicate (in some sheet there is twice PST )
    recommend_inter = recommend_inter.groupby(recommend_inter.index).first()
    #join the 2 dataframe, the name of the column are not important
    recommend = recommend.join(recommend_inter, how='outer', lsuffix='l', rsuffix='r')

recommend = recommend.transpose()  

#clean the dataframe 
recommend = recommend[~recommend.loc[:, 'Date'].str.contains('Parti 1').fillna(False)]
recommend = recommend[~recommend['Votation'].isnull()]
#create meaningfull index
recommend.index = range(len(recommend.index))

#transform the date into the same format than the main dataframe
month_to_int = { \
    'janvier' : '01', \
    'février' : '02', \
    'févirer' : '02', \
    'mars' : '03', \
    'avril' : '04', \
    'mai' : '05', \
    'juin' : '06', \
    'juillet' : '07', \
    'aout' : '08', \
    'septembre' : '09', \
    'octobre' : '10', \
    'novembre' : '11', \
    'décembre' : '12' \
}

def good_format_month (x) :
    if '.' in x :
        split_x = x.split('.')
        return split_x[0].zfill(2) + '.' + split_x[1].zfill(2) + '.' + split_x[2][:4]
    else :
        return str(x.split(' ')[0]).zfill(2) + '.' +  month_to_int[str(x.split(' ')[1]) ] + '.' + x[-4:] 

recommend.loc[:, 'Date'] = recommend.loc[:, 'Date'].map(lambda x : good_format_month(x))

recommend = recommend.loc[:, recommend.columns.drop_duplicates() ]
#parties is the list of all the parties
parties = list(recommend.columns.drop_duplicates())
parties.remove('Date')
parties.remove('Votation')
parties.remove('Parti')

recommend = recommend.loc[:, ['Date', 'Votation'] + parties]
recommend.head()

Change the value so that they are all uniform       
a yes recommandation becomes 1     
a no recommandation becomes -1   
No data or no recommendation is 0

In [ ]:
def translate_choice(x) :
    if (x == 1 or str(x) == 'oui') :
        return 1
    elif (x == 2 or str(x) == 'non'):
        return -1
    else :
        return 0

good_recommend = recommend.copy() 
good_recommend.loc[:, parties] = good_recommend.loc[:, parties].applymap(lambda x : translate_choice(x))
good_recommend

We need to create a dictionarry to link the name of the votation in the recomendation dataframe and the vote dataframe.   
The problem is that the name are very different, so to link one to another we need to group the votation of the 2 by date and for each date we compare all pair using SequenceMatch, we find the best match, we decide to link this pair together, we take out the 2 element from their respective list and we research a new maximum until there is no elemnt in one of the list.

This method might create some imprecision but as a whole it is solid.

In [ ]:
dico_recom_vote = {}

for date1, group1 in df[df['Commune'] == 'Aeugst am Albis'].groupby('Date'):
    
    for date2, group2 in good_recommend.groupby('Date'):
        
        if (date1 == date2) :
            vot1 = list(group1['Votation']) 
            vot2 = list(group2['Votation'])
            
            while (len(vot1) > 0 and len(vot2) > 0):
                max_v = -1
                max_match = [0, 0]
                
                for elem1 in vot1:
                    for elem2 in vot2:
                        current_v = SequenceMatcher(None, elem1, elem2).ratio()
                        
                        if (current_v > max_v) :
                            max_v = current_v
                            max_match = [elem1, elem2]
                            
                dico_recom_vote[max_match[0]] = max_match[1]
                dico_recom_vote[max_match[1]] = max_match[0]
                vot1.remove(max_match[0])
                vot2.remove(max_match[1])

Create a recommandation dataframe with names corresponding to the ones in the main dataframe

In [ ]:
final_recommand = good_recommend.copy()
final_recommand['Votation'] = final_recommand['Votation'].map(dico_recom_vote)
final_recommand.head()

In [ ]:
final_recommand.to_pickle('data/data_Recommandation.pkl')